## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-07-22-50-36 +0000,nypost,Cuban-led caravan en route to Mexico City as T...,https://nypost.com/2025/10/07/world-news/cuban...
1,2025-10-07-22-47-51 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/10/07/us/tru...
2,2025-10-07-22-38-56 +0000,nyt,Mass Layoffs Trump Is Threatening in Shutdown ...,https://www.nytimes.com/2025/10/07/us/politics...
3,2025-10-07-22-38-10 +0000,nyt,Health Care Politics Bolster Democrats in Shut...,https://www.nytimes.com/2025/10/07/us/politics...
4,2025-10-07-22-35-48 +0000,nypost,Manhattan-sized interstellar object 3I/ATLAS s...,https://nypost.com/2025/10/07/world-news/manha...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2327/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,38
16,shutdown,19
288,gaza,14
253,israel,12
292,deal,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
313,2025-10-06-22-57-48 +0000,nypost,Senate Democrats reject latest effort to end g...,https://nypost.com/2025/10/06/us-news/senate-d...,118
114,2025-10-07-18-11-39 +0000,nypost,Trump Middle East envoy won’t leave Egypt unti...,https://nypost.com/2025/10/07/world-news/kushn...,98
297,2025-10-07-01-27-00 +0000,wsj,President Trump signaled a willingness to stri...,https://www.wsj.com/politics/policy/trump-open...,96
124,2025-10-07-17-41-13 +0000,nypost,Trump says Dems ‘have no leader’ in shutdown s...,https://nypost.com/2025/10/07/us-news/trump-ri...,91
276,2025-10-07-06-39-28 +0000,nypost,"Trump to host Edan Alexander, American who spe...",https://nypost.com/2025/10/07/us-news/trump-to...,90


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
313,118,2025-10-06-22-57-48 +0000,nypost,Senate Democrats reject latest effort to end g...,https://nypost.com/2025/10/06/us-news/senate-d...
35,84,2025-10-07-21-22-46 +0000,nypost,Anti-Israel protesters chant antisemitic sloga...,https://nypost.com/2025/10/07/us-news/anti-isr...
139,50,2025-10-07-17-01-48 +0000,latimes,Supreme Court sees a free-speech problem with ...,https://www.latimes.com/politics/story/2025-10...
163,46,2025-10-07-15-50-55 +0000,nypost,NY woman refuses to pay $440 fine to Hertz aft...,https://nypost.com/2025/10/07/business/ny-woma...
114,46,2025-10-07-18-11-39 +0000,nypost,Trump Middle East envoy won’t leave Egypt unti...,https://nypost.com/2025/10/07/world-news/kushn...
38,42,2025-10-07-21-10-44 +0000,latimes,Sensitive police files to be opened up for wat...,https://www.latimes.com/california/story/2025-...
195,40,2025-10-07-14-01-04 +0000,cbc,"White House talks 'successful, positive, subst...",https://www.cbc.ca/news/politics/livestory/whi...
234,35,2025-10-07-10-00-00 +0000,nypost,Hostage Eli Sharabi was starved and beaten by ...,https://nypost.com/2025/10/07/world-news/eli-s...
103,32,2025-10-07-18-50-35 +0000,cbc,Gold prices hit record high as anxious investo...,https://www.cbc.ca/news/business/gold-prices-r...
158,30,2025-10-07-16-03-10 +0000,nypost,Pam Bondi lashes out at Dems in rowdy hearing ...,https://nypost.com/2025/10/07/us-news/pam-bond...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
